In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/public_notice/')
print(os.getcwd())

c:\Users\izard\Documents\GitHub\bimb-osint-development\back-end\bnm\public_notice


In [ ]:
driver = webdriver.Edge()

### Scrape Links for BNM Public Notice (English & Malay)

##### Get English Content URL

In [ ]:
#list of pages where links of available public notice #English

date=[]
title=[]
link=[]

for year in list(range(2009,2025)):
    url='https://www.bnm.gov.my/notices-announcements/-/tag/notices-'+str(year)
    print(url)
    driver.get(url)
    time.sleep(random.randint(5,10))
    soup=BeautifulSoup(driver.page_source)
    
    public_notice_list=soup.find('table',attrs={'class':'Press-table table'}).find_all('tr')[1:]

    for public_notice in public_notice_list:
        date.append(public_notice.find_all('p')[0].text)
        title.append(public_notice.find_all('p')[1].text)
        link.append(public_notice.find_all('p')[1])
            
df=pd.DataFrame({'date':date , 'title':title , 'link':link})
df['link']=df['link'].astype(str).str.extract('("https.+")')
df['link']=df['link'].apply(lambda x: x.replace('"',''))
df['link']=df['link'].str.replace(r'(>.+)','',regex=True)
df.to_csv('link1en.csv',index=False) # ref file 1

##### Get PDF (English) Links within Content

In [ ]:
link=[]
pdf_links=[]

df=pd.read_csv('link1en.csv')

for url in df['link'].to_list():
    print(url)
    link.append(url)
    driver = webdriver.Edge()
    driver.get(url)
    time.sleep(random.randint(5,10))
    soup=BeautifulSoup(driver.page_source)
    
    # Find all 'a' tags (which define hyperlinks)   
    a_tags = soup.find_all('a')
    
    file=[]
    for tag in a_tags:
        try:
            check_pdf=re.search('documents.+\.pdf', tag['href'])[0]
            check_pdf=check_pdf.replace('%2F','/') 
            #check_pdf in tag['href']
            file.append(check_pdf)
        except:
            pass
    
    file=list(set(file))
    
    # Filter the 'a' tags to get only those that link to a .pdf file
    pdf_links.append(file)
    
    df=pd.DataFrame({'link':link, 'pdf_links':pdf_links})
    df.to_csv('link-pdf-en.csv',index=False) # ref file 2
    
    driver.close()

##### Get Malay Content URL

In [ ]:
date=[]
title=[]
link=[]


for year in list(range(2024,2025)):
    url='https://www.bnm.gov.my/notices-announcements/-/tag/notices-'+str(year)
    #print(url)
    driver = webdriver.Edge()
    driver.get(url)
    
    time.sleep(random.randint(5,10))
    
    soup=BeautifulSoup(driver.page_source)
    
    link_bm = soup.find('a',attrs={'lang':'ms-MY'})
    link_bm = "https://www.bnm.gov.my"+(re.search('/.+languageId=ms_MY',str(link_bm)).group(0).replace('&amp;','&'))
    
    driver.get(link_bm)
    
    time.sleep(random.randint(5,10))
    
    soup=BeautifulSoup(driver.page_source)
    
    press_release_list=soup.find('table',attrs={'class':'Press-table table'}).find_all('tr')[1:]

    for press_release in press_release_list:
        date.append(press_release.find_all('p')[0].text)
        title.append(press_release.find_all('p')[1].text)
        link.append(press_release.find_all('p')[1])
        
    driver.close()

    
df=pd.DataFrame({'date':date , 'title':title , 'link':link})
df['link']=df['link'].astype(str).str.extract('("https.+")')
df['link']=df['link'].apply(lambda x: x.replace('"',''))
df['link']=df['link'].str.replace(r'(->.+)','-',regex=True)
df['link']=df['link'].str.replace(r'(>.+)','',regex=True)
df.to_csv('link1ms.csv',index=False) # ref file 3

#### Get PDF (Malay) Links within Content

In [ ]:
#link=[]
#pdf_links=[]

import pandas
df=pd.read_csv('link1ms.csv')
link=pd.read_csv('link-pdf-ms.csv')['link'].to_list()
pdf_links=pd.read_csv('link-pdf-ms.csv')['pdf_links'].to_list()

#for url in df['link'].to_list():

for url in tqdm([x for x in df['link'].to_list() if x not in link]):
    try:
        #print(url)
        link.append(url)
        driver = webdriver.Edge()
        driver.get(url)
        time.sleep(random.randint(5,10))
        soup=BeautifulSoup(driver.page_source)
        
        
        link_bm = soup.find('a',attrs={'lang':'ms-MY'})
        link_bm = "https://www.bnm.gov.my"+(re.search('/.+languageId=ms_MY',str(link_bm)).group(0).replace('&amp;','&'))
        print(link_bm)
        link_bm = link_bm.replace('%2F','/')
        
        driver.get(link_bm)
        time.sleep(random.randint(5,6))
        
        soup=BeautifulSoup(driver.page_source)
        
        # Find all 'a' tags (which define hyperlinks)   
        a_tags = soup.find_all('a')
        
        file=[]
        for tag in a_tags:
            try:
                check_pdf=re.search('documents.+\.pdf', tag['href'])[0]
                #print(check_pdf)
                check_pdf=check_pdf.replace('%2F','/') 
                #check_pdf in tag['href']
                file.append(check_pdf)
                print(file)
            except:
                continue
        
        file=list(set(file))
        
        # Filter the 'a' tags to get only those that link to a .pdf file
        pdf_links.append(file)
        
        df=pd.DataFrame({'link':link, 'pdf_links':pdf_links})
        df.to_csv('link-pdf-ms.csv',index=False) # ref file 4
        
        driver.close()
        
    except:
        
        print('failed at ',url)
        
        file=[]
        pdf_links.append(file)
        
        df=pd.DataFrame({'link':link, 'pdf_links':pdf_links})
        df.to_csv('link-pdf-ms.csv',index=False) # ref file 4
        
        driver.close()
        time.sleep(random.randint(5, 10))

    

#### Pdf links ref. table (Malay + English)

In [ ]:
pdf_links=pd.concat([pd.merge(pd.read_csv('link1ms.csv'),pd.read_csv('link-pdf-ms.csv'),how='left',on='link'),pd.merge(pd.read_csv('link1en.csv'),pd.read_csv('link-pdf-en.csv'),how='left',on='link')]).reset_index(drop=True)
pdf_links=pdf_links[~(pdf_links['link'].str.contains('%'))]
pdf_links=pdf_links.drop_duplicates('link').reset_index(drop=True)
pdf_links['link']=np.where(pdf_links['link'].str.endswith('.'),pdf_links['link'].str[:-1],pdf_links['link'])
pdf_links

In [ ]:
date=[]
title=[]
link=[]
links=[]

for x in list(range(len(pdf_links))):
    
    date.append(pdf_links['date'][x])
    title.append(pdf_links['title'][x])
    link.append(pdf_links['link'][x])
    
    try:
        
        pdf=""

        for row_link in re.search(r"'documents.+\.pdf'",pdf_links['pdf_links'][x])[0].replace("'","").split(","):
            pdf=pdf+'https://www.bnm.gov.my/'+row_link.strip()+", "

        pdf=pdf[:-2]
        
        links.append(pdf)
        
    except:
        
        pdf=""
        links.append(pdf)
    
df=pd.DataFrame({'date':date, 'title':title, 'link':link, 'pdf_links':links})

pdf_links=pd.merge(pdf_links.drop(columns=['pdf_links']), df, on=['date','title','link'], how='left')
pdf_links=pdf_links.drop_duplicates().reset_index(drop=True)
pdf_links



### Scrape Content Based on Links (English & Malay)

In [ ]:
files=os.listdir()
files=list(filter(lambda k: 'link1en' in k, files))

df=pd.DataFrame()

for file in files:
    df=pd.concat([df,(pd.read_csv(file))])
    df['link']=df['link'].str.replace(r'(>.+)','',regex=True)

#content=[]
#link=[]

#content=pd.read_csv('content1.csv')['content'].to_list()
#link=pd.read_csv('content1.csv')['link'].to_list()

for url in tqdm([x for x in df['link'].to_list() if x not in link]):
    try:
        try:
            print(url)
            driver = webdriver.Edge()
            driver.get(url)
            
            time.sleep(random.randint(5,6))
            
            soup=BeautifulSoup(driver.page_source)
            
            content.append(soup.find_all('div',attrs={'class':'journal-content-article'})[1].text)
            link.append(url)
            
            driver.close()
            
            dfContent=pd.DataFrame({'link':link , 'content':content})
            dfContent.to_csv('content1.csv', index=False) # ref file 5
            
        except:
            driver.close()
            driver = webdriver.Edge()
            driver.get(url)
            
            time.sleep(random.randint(5,6))
            
            soup=BeautifulSoup(driver.page_source)
            
            content.append(soup.find_all('div',attrs={'class':'journal-content-article'})[0].text)
            link.append(url)
            
            driver.close()
            
            dfContent=pd.DataFrame({'link':link , 'content':content})
            dfContent.to_csv('content1.csv', index=False) # ref file 5
            
            continue            
        
    except:
        
        print('failed at ',url)
        driver.close()
        time.sleep(random.randint(5, 10))
        
        continue
    

In [ ]:
data=pd.read_csv('link1ms.csv')
data['link']=data['link'].str.replace(r'(>.+)','',regex=True)

link=[]
link_opt=[]
content=[]

#link=link
#link_opt=link_opt
#content=content

for url in tqdm([x for x in data['link'].to_list() if x not in link]):
    try:
        #print(url)
        driver = webdriver.Edge()
        driver.get(url)
        
        time.sleep(random.randint(5,6))
        
        soup=BeautifulSoup(driver.page_source)
        
        
        link_bm = soup.find('a',attrs={'lang':'ms-MY'})
        link_bm = "https://www.bnm.gov.my"+(re.search('/.+languageId=ms_MY',str(link_bm)).group(0).replace('&amp;','&'))
        #print(link_bm)
        link_bm = link_bm.replace('%2F','/')
        
            
        driver.get(link_bm)
        
        time.sleep(random.randint(5,6))
        
        soup=BeautifulSoup(driver.page_source)
        #print(link_bm)
        #print(soup)
        content.append(soup.find_all('div',attrs={'class':'journal-content-article'})[1].text)
        link.append(url)
        link_opt.append(link_bm)
        
        driver.close()
        
        df=pd.DataFrame({'link':link , 'link_opt':link_opt , 'content':content})
        df.to_csv('content2.csv',index=False)

    except:
        print('failed at ',url)
        
        content.append(soup.find_all('div',attrs={'class':'journal-content-article'})[0].text)
        link.append(url)
        link_opt.append(link_bm)
        
        driver.close()
        
        df=pd.DataFrame({'link':link , 'link_opt':link_opt , 'content':content})
        df.to_csv('content2.csv',index=False)
        
        time.sleep(random.randint(10,20))
        continue




In [ ]:
#(pd.merge(pd.read_csv('link1en.csv'),pd.read_csv('content1.csv'),how='left',on='link')).to_csv('public_notice_english.csv',index=False)
#(pd.merge(pd.read_csv('link1ms.csv'),pd.read_csv('content2.csv'),how='left',on='link')).to_csv('public_notice_malay.csv',index=False)

#### Combine English & Malay Content + Add in Category Column as "Public Notice" 

In [ ]:
bnm_data=pd.concat([pd.read_csv('public_notice_english.csv'),pd.read_csv('public_notice_malay.csv')])
bnm_data['category']='Public Notice'
bnm_data=bnm_data.reset_index(drop=True)
bnm_data['link']=bnm_data['link'].str.strip()
bnm_data=bnm_data[~(bnm_data['link'].str.contains('%'))]
bnm_data=bnm_data.drop_duplicates('link')
bnm_data

In [ ]:
bnm_data['content']=bnm_data['content'].str.replace('\r\n\r\n\r\n\r\n','')
bnm_data['content']=bnm_data['content'].str.replace('\r\n\r\n\r\n','\r\n')
bnm_data['content']=bnm_data['content'].str.replace('\r\n\r\n\r\n\r\n\r\n\r\n\r\n','')
bnm_data['content']=bnm_data['content'].str.replace('Shares\r\n','')
bnm_data['content']=bnm_data['content'].str.replace('  Share:','Share:')
bnm_data['content']=bnm_data['content'].str.replace('      ','')
bnm_data



In [ ]:
bnm_data=pd.merge(pdf_links,bnm_data,how='left',left_on=['date','title','link'],right_on=['date','title','link']).drop_duplicates()
bnm_data


### Download PDF files

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
import pyautogui
import pyperclip


link = []
pdf_links = []

# Filter the DataFrame once and reset the index

'''
filtered_data = bnm_data[bnm_data['pdf_links'] != ''].reset_index(drop=True)
downloaded=pd.DataFrame({'link':link,'pdf_links':pdf_links})[['link']].drop_duplicates()
downloaded['tag']='yes'
filtered_data=pd.merge(filtered_data,downloaded,how='left',on='link')
filtered_data=filtered_data[filtered_data['tag'].isnull()]
filtered_data=filtered_data.drop(columns=['tag'])
filtered_data=filtered_data.reset_index(drop=True)
'''

filtered_data = bnm_data[bnm_data['pdf_links'] != ''].reset_index(drop=True)
filtered_data = filtered_data[['link','pdf_links']]
filtered_data['pdf_links']=(filtered_data['pdf_links'].str.split(', ',expand=False))
filtered_data = filtered_data.explode('pdf_links')
filtered_data['filename']=filtered_data['pdf_links'].str.split('/').str[-1]
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/public_notice/pdf/')
filtered_data=pd.merge(filtered_data,pd.DataFrame({'filename':os.listdir(),'downloaded':'yes'}),how='left',on='filename')
filtered_data=filtered_data[filtered_data['downloaded'].isnull()].drop(columns=['downloaded'])
filtered_data=filtered_data.reset_index(drop=True)

# Setup Edge options
edge_options = Options()

# Initialize the driver once
driver = webdriver.Edge(options=edge_options)

for row in range(len(filtered_data)):
    current_link = filtered_data['link'][row]
    pdf_list = filtered_data['pdf_links'][row].split(', ')
    
    for pdf in pdf_list:
        
        try:
        
            link.append(current_link)
            pdf_links.append(pdf)
            driver.get(pdf)
            
            time.sleep(30)  # Wait for the page to load
            pyautogui.hotkey('ctrl', 's')  # Simulate Ctrl + S (save)
            time.sleep(10)  # Wait for the save dialog to appear
            path_and_filename = 'c:\\Users\\izard\\Documents\\GitHub\\bimb-osint-development\\back-end\\bnm\\public_notice\\pdf\\'+pdf.split('/')[-1]
            pyperclip.copy(path_and_filename)
            pyautogui.hotkey('ctrl', 'v')
            pyautogui.press('enter')  # Press Enter to save the file
            pyautogui.press('enter')  # Press Enter to save the file
            
            time.sleep(random.randint(10,20))
            
            print('success')
        
        except:
            print('unsuccess')
            pass

# Close the driver after processing all rows
driver.quit()


### PDF to Text

In [ ]:
pdf_files = bnm_data[bnm_data['pdf_links'] != ''].reset_index(drop=True)
pdf_files = pdf_files[['link','pdf_links']]
pdf_files['pdf_links']=(pdf_files['pdf_links'].str.split(', ',expand=False))
pdf_files = pdf_files.explode('pdf_links')
pdf_files['filename']=pdf_files['pdf_links'].str.split('/').str[-1]
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/public_notice/pdf/')
pdf_files=pd.merge(pdf_files,pd.DataFrame({'filename':os.listdir(),'downloaded':'yes'}),how='left',on='filename')
pdf_files=pdf_files[pdf_files['downloaded'].notnull()].drop(columns=['downloaded'])
pdf_files=pdf_files.reset_index(drop=True)
pdf_files

In [ ]:
# start tika server , check at http://localhost:9998/

os.chdir('/Users/izard/') # change directory to where tika-server-standard-2.6.0.jar is located

! java -jar tika-server-standard-2.6.0.jar

In [ ]:
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/public_notice/pdf/')
os.getcwd()

In [24]:
import tika
from tika import parser
from tqdm import tqdm

downloaded_files=os.listdir() # get list of downloaded files

tika.initVM()
print('start..')

filename=[]
Document_Text=[]

for file in tqdm(downloaded_files):

    try:
        print('parsing...',file)
        
        text=parser.from_file(file)['content']
        
        filename.append(file)
        Document_Text.append(text)
                
    except:
        print('unsuccess...',file)
        filename.append(file)
        Document_Text.append('')
        pass
    
pdf_to_text=pd.DataFrame({'filename':filename, 'Document_Text':Document_Text})

 16%|█▋        | 66/405 [04:26<1:56:20, 20.59s/it]

unsuccess... Appendix+I+-+adjuster-regnreqmt.pdf
parsing... Appendix+II+-+adjuster-glines-policies.pdf


 17%|█▋        | 67/405 [04:28<1:24:41, 15.03s/it]

parsing... Appendix+III+-+adjuster-registrationform.pdf


 17%|█▋        | 68/405 [04:30<1:02:36, 11.15s/it]

...

100%|█████████▉| 403/405 [24:58<00:05,  2.60s/it]

parsing... Updated+22062020+Financial+Consumer+Alert%27s+List+%28Eng%29.pdf


100%|█████████▉| 404/405 [25:00<00:02,  2.45s/it]

parsing... WMC_Guide.pdf


100%|██████████| 405/405 [25:03<00:00,  3.71s/it]


In [34]:
print(pdf_to_text[pdf_to_text['filename']=='cba2009_02.pdf']['Document_Text'].iloc[0])








































514539 BM.indd


Bank Negara Malaysia 1

 

UNDANG-UNDANG
MALAYSIA

Akta 701

AKTA BANK NEGARA MALAYSIA 2009



Undang-Undang Malaysia2 AKTA 701

Tarikh Perkenan Diraja ... ... 19 Ogos 2009

Tarikh penyiaran dalam Warta  ... 3 September 2009



Bank Negara Malaysia 3

UNDANG-UNDANG MALAYSIA

Akta 701

AKTA BANK NEGARA MALAYSIA 2009

SUSUNAN SEKSYEN

BAHAGIAN I

PERMULAAN

 Seksyen

 1. Tajuk ringkas dan permulaan kuat kuasa

 2. Tafsiran

BAHAGIAN II

BANK, MATLAMAT DAN FUNGSINYA

 3. Bank yang ditubuhkan di bawah Akta Bank Negara Malaysia 1958

 4. Bank pusat bagi Malaysia

 5. Matlamat dan fungsi utama Bank

BAHAGIAN III

PERUNTUKAN KEWANGAN

 6. Modal

 7. Kumpulan Wang Rizab Am

 8. Tahun kewangan

 9. Kehendak untuk menyimpan akaun dan rekod lain

 10. Piawaian perakaunan

 11. Audit luar

 12. Penyata aset dan liabiliti

 13. Penyediaan dan penerbitan penyata kewangan dan laporan 



Undang-Undang Malaysia4 AKTA 701

BAHAGIAN IV

TADBIR URUS

#### Update pdf_files with text extracted columns

In [41]:
pdf_files=pd.merge(pdf_files,pdf_to_text,how='left',on='filename')
pdf_files

,link,pdf_links,filename,Document_Text
0,https://www.bnm.gov.my/-/akta-bank-negara-mala...,https://www.bnm.gov.my/documents/20124/761712/...,cba2009_02.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,https://www.bnm.gov.my/-/jadual-perbandingan-b...,https://www.bnm.gov.my/documents/20124/761712/...,FAQ_on_Pembiayaan_Mikro_BM.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,https://www.bnm.gov.my/-/pembiayaan-mikro-jadu...,https://www.bnm.gov.my/documents/20124/761712/...,Comparative_table_on_Pembiayaan_Mikro_BM.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,https://www.bnm.gov.my/-/pembiayaan-mikro-jadu...,https://www.bnm.gov.my/documents/20124/761712/...,FAQ_on_Pembiayaan_Mikro_BM.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,https://www.bnm.gov.my/-/terbitan-terbaharu-bu...,https://www.bnm.gov.my/documents/20124/761709/...,shariah_resolutions_2nd_edition.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...,...,...
581,https://www.bnm.gov.my/-/jc3-sc1-ccpt-docs,https://www.bnm.gov.my/documents/20124/3770663...,Guidance-Notes-DDQ-GP3-GP4-10Jan24.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
582,https://www.bnm.gov.my/-/ed-spm,https://www.bnm.gov.my/documents/20124/938039/...,ed_Skim_Pembiayaan_Mikro_Policy.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
583,https://www.bnm.gov.my/-/sacbnm-226mtg-ruling,https://www.bnm.gov.my/documents/20124/38335/s...,sacbnm-mtg-226.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
584,https://www.bnm.gov.my/-/pd-hajah-darurah-en,https://www.bnm.gov.my/documents/20124/938039/...,pd-Hajah-Darurah-Jan2024.pdf,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


#### Filter Only to PDF to Text Valid Rows

In [51]:
pdf_files['Document_Text']=np.where((pdf_files['Document_Text']=='')|(pdf_files['Document_Text'].isnull()),'N/A',pdf_files['Document_Text'])

#### Concat PDF to Text Extracted with Same Link

In [73]:
pdf_files=pdf_files[['link','Document_Text']].groupby(['link']).agg({'Document_Text':'sum'}).reset_index()
pdf_files

,link,Document_Text
0,https://www.bnm.gov.my/-/2nd-fin-incl-frmwk,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
1,https://www.bnm.gov.my/-/2nd-fin-incl-frmwk-bm,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
2,https://www.bnm.gov.my/-/4q-2017-labour-market...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
3,https://www.bnm.gov.my/-/8th-mystats-2021,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
4,https://www.bnm.gov.my/-/academic-visit-calend...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
...,...,...
450,https://www.bnm.gov.my/-/unauthorised-company-...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
451,https://www.bnm.gov.my/-/unauthorised-company-...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
452,https://www.bnm.gov.my/-/unauthorised-company-...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...
453,https://www.bnm.gov.my/-/wholesale-market-cond...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...


#### Update Main Dataframe

In [75]:
pd.merge(bnm_data,pdf_files,how='left',on='link').columns

Index(['date', 'title', 'link', 'pdf_links', 'content', 'link_opt', 'category',
       'Document_Text'],
      dtype='object')

In [78]:
bnm_data=pd.merge(bnm_data,pdf_files,how='left',on='link')[['date', 'title', 'link', 'pdf_links', 'link_opt', 'content', 'Document_Text', 'category']]
os.chdir('/Users/izard/Documents/GitHub/bimb-osint-development/back-end/bnm/public_notice/')
bnm_data.to_csv('bnm_data_public_notice.csv',index=False)
bnm_data

,date,title,link,pdf_links,link_opt,content,Document_Text,category
0,28 Dis 2009,Concept Paper on Shariah Parameter Reference 3...,https://www.bnm.gov.my/-/concept-paper-3-mudar...,,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nConcept Paper on Shariah P...,NaN,Public Notice
1,03 Dis 2009,Concept Paper - Guidelines on Takaful Operatio...,https://www.bnm.gov.my/-/concept-paper-takaful...,,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nConcept Paper - Guidelines...,NaN,Public Notice
2,25 Nov 2009,Akta Bank Negara Malaysia 2009,https://www.bnm.gov.my/-/akta-bank-negara-mala...,https://www.bnm.gov.my/documents/20124/761712/...,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nAkta Bank Negara Malaysia ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
3,17 Nov 2009,Senarai Penasihat Kewangan telah dikemaskini. ...,https://www.bnm.gov.my/-/financial-adviser-lis...,,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nSenarai Penasihat Kewangan...,NaN,Public Notice
4,13 Nov 2009,Sertai kajian Tahap Kepuasan Pengguna laman we...,https://www.bnm.gov.my/-/sertai-kajian-tahap-k...,,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1099,17 Jan 2024,Exposure Draft on Currency Processing Business,https://www.bnm.gov.my/-/ed-cpb,https://www.bnm.gov.my/documents/20124/1320456...,NaN,\r\n\r\nReading:\r\nExposure Draft on Currency...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
1100,12 Jan 2024,JC3 SC1 Updates Documents for Implementation o...,https://www.bnm.gov.my/-/jc3-sc1-ccpt-docs,https://www.bnm.gov.my/documents/20124/3770663...,NaN,\r\n\r\nReading:\r\nJC3 SC1 Updates Documents ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
1101,08 Jan 2024,Exposure Draft on Skim Pembiayaan Mikro,https://www.bnm.gov.my/-/ed-spm,https://www.bnm.gov.my/documents/20124/938039/...,NaN,\r\n\r\nReading:\r\nExposure Draft on Skim Pem...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
1102,05 Jan 2024,BNM Shariah Advisory Council's Ruling on Cedin...,https://www.bnm.gov.my/-/sacbnm-226mtg-ruling,https://www.bnm.gov.my/documents/20124/38335/s...,NaN,\r\n\r\nReading:\r\nBNM Shariah Advisory Counc...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice


#### Convert to JSON

In [2]:
# standardize the timestamp

from datetime import datetime,timedelta

bnm_data=pd.read_csv('bnm_data_public_notice.csv')
bnm_data['date']=np.where(bnm_data['date'].str.contains('Dis'),bnm_data['date'].str.replace('Dis','Dec'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Okt'),bnm_data['date'].str.replace('Okt','Oct'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Ogos'),bnm_data['date'].str.replace('Ogos','Aug'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Mei'),bnm_data['date'].str.replace('Mei','May'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Mac'),bnm_data['date'].str.replace('Mac','Mar'),bnm_data['date'])
bnm_data['date']=np.where(bnm_data['date'].str.contains('Mac'),bnm_data['date'].str.replace('Mac','Mar'),bnm_data['date'])
bnm_data['date']=pd.to_datetime(bnm_data['date'])

In [5]:
bnm_data=bnm_data.rename(columns={'date':'Date_created','title':'Title','link':'URL','pdf_links':'pdf_URL','link_opt':'URL_opt','content':'Content','category':'Category'})
bnm_data

,Date_created,Title,URL,pdf_URL,URL_opt,Content,Document_Text,Category
0,2009-12-28,Concept Paper on Shariah Parameter Reference 3...,https://www.bnm.gov.my/-/concept-paper-3-mudar...,NaN,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nConcept Paper on Shariah P...,NaN,Public Notice
1,2009-12-03,Concept Paper - Guidelines on Takaful Operatio...,https://www.bnm.gov.my/-/concept-paper-takaful...,NaN,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nConcept Paper - Guidelines...,NaN,Public Notice
2,2009-11-25,Akta Bank Negara Malaysia 2009,https://www.bnm.gov.my/-/akta-bank-negara-mala...,https://www.bnm.gov.my/documents/20124/761712/...,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nAkta Bank Negara Malaysia ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
3,2009-11-17,Senarai Penasihat Kewangan telah dikemaskini. ...,https://www.bnm.gov.my/-/financial-adviser-lis...,NaN,https://www.bnm.gov.my/c/portal/update_languag...,\r\n\r\nReading:\r\nSenarai Penasihat Kewangan...,NaN,Public Notice
4,2009-11-13,Sertai kajian Tahap Kepuasan Pengguna laman we...,https://www.bnm.gov.my/-/sertai-kajian-tahap-k...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1099,2024-01-17,Exposure Draft on Currency Processing Business,https://www.bnm.gov.my/-/ed-cpb,https://www.bnm.gov.my/documents/20124/1320456...,NaN,\r\n\r\nReading:\r\nExposure Draft on Currency...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
1100,2024-01-12,JC3 SC1 Updates Documents for Implementation o...,https://www.bnm.gov.my/-/jc3-sc1-ccpt-docs,https://www.bnm.gov.my/documents/20124/3770663...,NaN,\r\n\r\nReading:\r\nJC3 SC1 Updates Documents ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
1101,2024-01-08,Exposure Draft on Skim Pembiayaan Mikro,https://www.bnm.gov.my/-/ed-spm,https://www.bnm.gov.my/documents/20124/938039/...,NaN,\r\n\r\nReading:\r\nExposure Draft on Skim Pem...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice
1102,2024-01-05,BNM Shariah Advisory Council's Ruling on Cedin...,https://www.bnm.gov.my/-/sacbnm-226mtg-ruling,https://www.bnm.gov.my/documents/20124/38335/s...,NaN,\r\n\r\nReading:\r\nBNM Shariah Advisory Counc...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Public Notice


In [6]:
out=bnm_data.to_json(orient = "records",date_format='iso')

with open('bnm_data_public_notice.json', 'w') as f:
       f.write(out)

In [25]:
os.getcwd()    

'c:\\Users\\izard\\Documents\\GitHub\\bimb-osint-development\\back-end\\bnm\\public_notice'